## Import Packages

In [10]:
import csv
import numpy as np
from sklearn.cross_validation import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

## Extract Data

In [11]:
def findCommunities(filename):
    communities = []
    with open(filename, 'r') as csvfile:
        filereader = csv.reader(csvfile)
        j = 0
        for i in filereader:
            if j > 0:
                if i[1] not in communities:
                    communities.append(i[1])
            j+=1
    return communities
def extractData(filename,communities,addstate = False,addcommunity = False):
    features = []
    xPositive=[]
    yPositive=[]
    xNegative=[]
    yNegative =[]
    with open(filename, 'r') as csvfile:
        filereader = csv.reader(csvfile)
        j = 0
        for i in filereader:
            if j > 0:
                data = i[3:len(i)-1]
                if addstate:
                    state = [0.0]*56
                    state[int(i[0])-1] = 1.0
                    data = data + state
                    features = features + ["state"+str(i) for i in range(56)]
                if addcommunity:
                    tmp = [0.0]*len(communities)
                    tmp[communities.index(i[1])] = 1.0
                    data = data + tmp
                    features = features + communities
                vect = np.array(data)
                #vect = np.array(i[3:len(i)-1])
                #x.append(vect.astype(np.float))
                if float(i[-1]) > 0.1:
                    xPositive.append(vect.astype(np.float))
                    yPositive.append(1)
                else:
                    xNegative.append(vect.astype(np.float))
                    yNegative.append(0)
            else:
                features = i[3:len(i)-1]
            j+=1
        """yPositive = np.array(yPositive)
        xPositive = np.array(xPositive)
        yNegative = np.array(yNegative)
        xNegative = np.array(xNegative)"""
        features = np.array(features)
        indxP = int(len(xPositive)*.6)
        indxN = int(len(xNegative)*.6)
        xTrain =np.array(xPositive[:indxP] + xNegative[:indxN])
        yTrain = np.array(yPositive[:indxP] + yNegative[:indxN])
        xTest = np.array(xPositive[indxP:] + xNegative[indxN:])
        yTest = np.array(yPositive[indxP:] + yNegative[indxN:])
        
        
            
            
        return xTrain,yTrain,xTest,yTest,features

## Calculate Label Percentage

In [12]:
def calcualtePercentage(labels):
    n = len(labels)
    sumTrue = sum(labels)*1.0
    sumFalse = (n - sumTrue)*1.0
    return sumTrue/n,sumFalse/n

## Create Model

In [13]:
def randomForest():
    return RandomForestClassifier.RandomForestClassifier(n_estimators=10, criterion='gini', max_depth=None, min_samples_split=2, 
                                       min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', 
                                       max_leaf_nodes=None, min_impurity_split=1e-07, bootstrap=True, oob_score=False, 
                                       n_jobs=1, random_state=None, verbose=0, warm_start=False, class_weight=None)

## Evaluation

In [14]:
def confusionMatrix(true,predicted,labels = [1,0]):
    return confusion_matrix(true,predicted,labels)

def find_confusion_matrix(actual,predicted,clabels):
    cm= []
    for i in clabels:
        tmp =[0]*len(clabels)
        
        for j in range(len(actual)):
            if actual[j]== i and actual[j] == predicted[j]:
                tmp[clabels.index(i)] += 1
            elif actual[j]== i and actual[j] != predicted[j]:
                tmp[clabels.index(predicted[j])] += 1
        cm.append(tmp)
    return np.array(cm)
def find_accuracy(matrix):
    return np.trace(matrix)*1.0/np.sum(matrix)
def find_precision(matrix):
    pres = []
    x = np.sum(matrix,axis=0)
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if i == j:
                pres.append(matrix[i][j]*1.0/x[i])
    return pres
def find_recall(matrix):
    rec = []
    x = np.sum(matrix,axis=1)
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if i == j:
                rec.append(matrix[i][j]*1.0/x[i])
    return rec
def evaluation(acutal,predicted,clabels=[1,0]):
    confmatrix = find_confusion_matrix(acutal,predicted,clabels)
    print ("Confusion Matrix")
    print (confmatrix)
    accuracy = find_accuracy(confmatrix)
    print ("Accuracy", accuracy)
    precision = find_precision(confmatrix)
    print ("Precision", precision)
    recall = find_recall(confmatrix)
    print ("Recall", recall)

## Cross Validation

In [15]:
def do_cross_validation(X, y,clf, n_folds=5):
    cv = KFold(len(y), n_folds, shuffle=True)
    accuracies = []
    i = 1
    for train_ind, test_ind in cv: 
        clf.fit(X[train_ind], y[train_ind])
        predictions = clf.predict(X[test_ind])
        print ("Fold %d Evaluation" %i)
        i +=1
        evaluation(y[test_ind],predictions)
        print ("\n")
        accuracies.append(accuracy_score(y[test_ind], predictions))
    avg = np.mean(accuracies)
    print ("The average Accuracy is ", avg)

### Read Data

In [16]:
filename = 'C:\\Users\\ar1\\Documents\ML\\Project\\Crime Prediction Data(1)\\Crime Prediction Data\\communities-crime-clean.csv'
distintCommunities = findCommunities(filename)
XTrain,YTrain,XTest,YTest,Features = extractData(filename, distintCommunities)

### Percentage of Labels

In [17]:
print ("Train Percentage",calcualtePercentage(YTrain))
print ("Test Percentage",calcualtePercentage(YTest))

Train Percentage (0.62761506276150625, 0.3723849372384937)
Test Percentage (0.62656641604010022, 0.37343358395989973)


### Default Decision Tree Model

In [18]:
model= RandomForestClassifier(max_depth=None, min_samples_split=2,n_estimators=10)
model.fit(XTrain,YTrain)
predictedY=model.predict(XTest)
evaluation(YTest,predictedY)

Confusion Matrix
[[399 101]
 [ 69 229]]
Accuracy 0.786967418546
Precision [0.85256410256410253, 0.69393939393939397]
Recall [0.79800000000000004, 0.76845637583892612]


### 10 Fold Cross Validation on Default DecisionTree Model

In [19]:
do_cross_validation(XTrain,YTrain,model,10)

Fold 1 Evaluation
Confusion Matrix
[[62 10]
 [12 36]]
Accuracy 0.816666666667
Precision [0.83783783783783783, 0.78260869565217395]
Recall [0.86111111111111116, 0.75]


Fold 2 Evaluation
Confusion Matrix
[[66 14]
 [ 4 36]]
Accuracy 0.85
Precision [0.94285714285714284, 0.71999999999999997]
Recall [0.82499999999999996, 0.90000000000000002]


Fold 3 Evaluation
Confusion Matrix
[[62 11]
 [ 9 38]]
Accuracy 0.833333333333
Precision [0.87323943661971826, 0.77551020408163263]
Recall [0.84931506849315064, 0.80851063829787229]


Fold 4 Evaluation
Confusion Matrix
[[57 13]
 [10 40]]
Accuracy 0.808333333333
Precision [0.85074626865671643, 0.75471698113207553]
Recall [0.81428571428571428, 0.80000000000000004]


Fold 5 Evaluation
Confusion Matrix
[[56 10]
 [ 8 46]]
Accuracy 0.85
Precision [0.875, 0.8214285714285714]
Recall [0.84848484848484851, 0.85185185185185186]


Fold 6 Evaluation
Confusion Matrix
[[69  9]
 [ 9 32]]
Accuracy 0.848739495798
Precision [0.88461538461538458, 0.78048780487804881]
Reca

### Top 10 important features 

In [20]:
coef = model.feature_importances_
topindx = np.argsort(coef)[::-1][:10]
print (Features[topindx])


['racePctWhite' 'PctKids2Par' 'PctIlleg' 'PctPersDenseHous' 'NumIlleg'
 'TotalPctDiv' 'PctYoungKids2Par' 'pctWInvInc' 'medFamInc' 'PctHousOwnOcc']
